## Document Clustering for People Wiki

In [1]:
import pandas as pd
people = pd.read_csv(f"D:/Docs/people_wiki.csv")
#people = pd.read_csv(f"D:/SYED/data/people_wiki.csv")
people.head(3)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...


In [2]:
obama = people[people['name'] == 'Barack Obama']
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(obama['text'])

In [3]:
counts = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())
counts.T.sort_values(by=0, ascending=False).head(10)

,0
the,40
in,30
and,21
of,18
to,14
his,11
obama,9
act,8
he,7
as,6


In [4]:
pep1 = people[0:100]
pep1.head(3)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...


## Function to make word-count as dict

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
def make_counts(text_data):
    cv_fit = cv.fit_transform(text_data)
    word_list = cv.get_feature_names_out()
    count_list = np.asarray(cv_fit.sum(axis=0))[0]
    final_dict = dict(zip(word_list, count_list))
    return final_dict

In [6]:
pep1['word_count'] = pep1['text'].apply(lambda x: make_counts([x]))
pep1.head(5)

<ipython-input-6-dca946018991>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pep1['word_count'] = pep1['text'].apply(lambda x: make_counts([x]))


,URI,name,text,word_count
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'10': 1, '1979': 1, '19982000': 1, '2000': 1,..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'1973': 1, '1981': 1, '2005': 1, '24hour': 1,..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'1982': 1, '1995': 1, '2010': 1, '2014': 2, '..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'18': 1, '1942': 1, '1963': 1, '1969': 1, '19..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'18': 1, '1974': 1, '1996': 1, '1998': 1, '20..."


## TF-IDF Calculation

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words='english')
tfidf_term_matrix = v.fit_transform(pep1.text)
df1 = pd.DataFrame(tfidf_term_matrix.toarray(), columns = v.get_feature_names_out())
df1.shape

(100, 6728)

In [8]:
df3 = pep1.drop(pep1.iloc[:, 5:6728],axis = 1)
df3.head(5)

,URI,name,text,word_count
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'10': 1, '1979': 1, '19982000': 1, '2000': 1,..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'1973': 1, '1981': 1, '2005': 1, '24hour': 1,..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'1982': 1, '1995': 1, '2010': 1, '2014': 2, '..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'18': 1, '1942': 1, '1963': 1, '1969': 1, '19..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'18': 1, '1974': 1, '1996': 1, '1998': 1, '20..."


In [9]:
tfidf_dense = tfidf_term_matrix.todense()

In [10]:
new_cols = v.get_feature_names()

C:\Users\SYED\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
df4 = pd.DataFrame(tfidf_dense, columns = new_cols)

In [13]:
df4.head(25)

,10,100,1000,102,109,10part,10th,11,110,1115,...,zines,zionism,zlata,zone,zones,zoubeir,zulu,zurich,zvonimir,zwigoff
0,0.045127,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
6,0.024879,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7,0.000000,0.0,0.050362,0.000000,0.0,0.0,0.0,0.039121,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
8,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
9,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
